## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,48.31,87,75,8.05,light rain
1,1,Mataura,NZ,-46.1927,168.8643,56.48,98,100,5.79,overcast clouds
2,2,Codrington,AU,-38.2667,141.9667,58.50,62,99,12.19,overcast clouds
3,3,Richards Bay,ZA,-28.7830,32.0377,79.09,77,23,15.88,few clouds
4,4,Muncar,ID,-8.4333,114.3333,75.54,90,78,3.49,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 10
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,48.31,87,75,8.05,light rain
1,1,Mataura,NZ,-46.1927,168.8643,56.48,98,100,5.79,overcast clouds
2,2,Codrington,AU,-38.2667,141.9667,58.50,62,99,12.19,overcast clouds
3,3,Richards Bay,ZA,-28.7830,32.0377,79.09,77,23,15.88,few clouds
4,4,Muncar,ID,-8.4333,114.3333,75.54,90,78,3.49,broken clouds
6,6,Severo-Kurilsk,RU,50.6789,156.1250,26.85,88,96,7.20,overcast clouds
8,8,Hermanus,ZA,-34.4187,19.2345,70.81,77,14,20.22,few clouds
10,10,Margate,GB,51.3813,1.3862,51.89,76,94,1.01,overcast clouds
11,11,Hamilton,US,39.1834,-84.5333,45.07,93,100,1.01,moderate rain
12,12,Dandong,CN,40.1292,124.3947,12.56,76,0,3.09,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Punta Arenas,CL,48.31,light rain,-53.1500,-70.9167,
1,Mataura,NZ,56.48,overcast clouds,-46.1927,168.8643,
2,Codrington,AU,58.50,overcast clouds,-38.2667,141.9667,
3,Richards Bay,ZA,79.09,few clouds,-28.7830,32.0377,
4,Muncar,ID,75.54,broken clouds,-8.4333,114.3333,
6,Severo-Kurilsk,RU,26.85,overcast clouds,50.6789,156.1250,
8,Hermanus,ZA,70.81,few clouds,-34.4187,19.2345,
10,Margate,GB,51.89,overcast clouds,51.3813,1.3862,
11,Hamilton,US,45.07,moderate rain,39.1834,-84.5333,
12,Dandong,CN,12.56,clear sky,40.1292,124.3947,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        continue
    

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace(r'^s*$',float('Nan'),regex = True)
clean_hotel_df = clean_hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))